In [ ]:
import networkx as nx
import pylab
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets
import bqplot

In [2]:
def mod_pert_random(low, likely, high, confidence=4, samples=10000):
    """Produce random numbers according to the 'Modified PERT'
    distribution.
    :param low: The lowest value expected as possible.
    :param likely: The 'most likely' value, statistically, the mode.
    :param high: The highest value expected as possible.
    :param confidence: This is typically called 'lambda' in literature
                        about the Modified PERT distribution. The value
                        4 here matches the standard PERT curve. Higher
                        values indicate higher confidence in the mode.
                        Currently allows values 1-18
    :param samples: random number size
    Formulas from "Modified Pert Simulation" by Paulo Buchsbaum.
    """
    # Check minimum & maximum confidence levels to allow:
    if confidence < 1 or confidence > 18:
        raise ValueError('confidence value must be in range 1-18.')

    mean = (low + confidence * likely + high) / (confidence + 2)

    a = (mean - low) / (high - low) * (confidence + 2)
    b = ((confidence + 1) * high - low - confidence * likely) / (high - low)

    beta = np.random.beta(a, b, samples)
    beta = beta * (high - low) + low
    return beta

In [3]:
def mapping(length, width):
    """
    With designated length and width, this function generate a length x width grid. And define the middle point as
    the location of restaurant.
    :param length: The number of nodes vertically
    :param width: The number of nodes horizontally
    :return: A NetworkX 2D gird graph
    """
    G = nx.grid_2d_graph(length, width)
    for edge in list(G.edges):
        low = np.random.randint(1, 10)
        high = np.random.randint(low + 1, 30)
        likely = np.random.randint(low, high)
        G.edges[edge[0], edge[1]]['low']=low
        G.edges[edge[0], edge[1]]['high']=high
        G.edges[edge[0], edge[1]]['likely']=likely
    G.nodes[int(length / 2), int(width / 2)]['name'] = 'RESTAURANT'
    return G

In [4]:
def real_map(G):
    for edge in list(G.edges):
        low= G.edges[edge[0], edge[1]]['low']
        high= G.edges[edge[0], edge[1]]['high']
        likely=G.edges[edge[0], edge[1]]['likely']
        distribution=mod_pert_random(low, likely, high, confidence=2)
        number=np.random.randint(0,len(distribution))
        G.edges[edge[0], edge[1]]['time'] = distribution[number]
    return G

In [5]:
def prep_time(order_size):
    """
    Return the preparation time for the order based on its size
    :param order_size: size for the order
    :return: preparation time
    """
    preparation_time = np.random.uniform(0, 10, size=100)
    if order_size == "S":
        return preparation_time
    elif order_size == "M":
        return preparation_time * 1.5
    else:
        return preparation_time * 2

In [6]:
def deliver_wait_time(queue_count):
    """
    Return the wait time of the deliver man for this specific order
    :param: none
    :return: wait time
    """
    wait_time = 0
    # When the order arrives, the total orders in the queue,maximum 10
    order_size = ['S', 'M', 'L']
    for i in range(queue_count + 1):
        order_num = np.random.randint(0, 3)
        preparation_time = prep_time(order_size[order_num])
        number = np.random.randint(0, len(preparation_time))
        wait_time += preparation_time[number]
    return wait_time

In [7]:
def judgement(mapgrid, new_delivery_point,queue_count,previous_order_location):
    """
    Depend on the time that cost on wait and traffic to new delivery point, this function tells if we should wait or
    leave now
    :param mapgrid: Made grid with restaurant location
    :param new_delivery_point: Randomly generated location on grid
    :param queue_count:how many orders are waiting to be prepared when the new order arrives
    :param previous_order_location: last order location 
    :return: 'W'ait or leave 'N'ow
    """
    coordinate = (0, 0)
    for restaurant_location, name in nx.get_node_attributes(mapgrid, 'name').items():
        if name == 'RESTAURANT':
            coordinate = restaurant_location
    new_time = nx.dijkstra_path_length(mapgrid, source=coordinate, target=new_delivery_point, weight='time')
    previous_time = nx.dijkstra_path_length(mapgrid, source=coordinate, target=previous_order_location, weight='time')
    time_two_destination = nx.dijkstra_path_length(mapgrid, source=previous_order_location, target=new_delivery_point,
                                                   weight='time')
    if (new_time + previous_time) * 2 > deliver_wait_time(queue_count) + new_time + time_two_destination + previous_time:
        return {new_delivery_point: 'W'}  # for 'wait'
    else:
        return {new_delivery_point: 'N'}  # for 'now'

In [132]:
if __name__ == '__main__':
    location_list = []
    decision_list = []
    length = int(input('Please input the length of the grid:\n'))
    width = int(input('Please input the width of the grid:\n'))
    queue_count=int(input('Please input how many orders are waiting before this order:\n'))
    coordinates=input('Please input previous order location:example:1,1\n')
    x=int(coordinates.split(',')[0])
    y=int(coordinates.split(',')[1])
    previous_order_location=(x,y)
    generate_map = mapping(length, width)
    for repeat in list(range(1000)):
        new_map=real_map(generate_map)
        for nodes in list(new_map.nodes()):
            for location, decision in judgement(new_map, nodes,queue_count,previous_order_location).items():
                location_list.append(location)
                decision_list.append(decision)
    result = pd.DataFrame({'location': location_list, 'decision': decision_list})
    temp = result.groupby('location')['decision'].sum().reset_index()
    list_N = []
    list_W = []
    for i in list(range(length * width)):
        n_N = temp['decision'][i].count('N')
        n_W = temp['decision'][i].count('W')
        list_N.append(n_N)
        list_W.append(n_W)
    result = pd.DataFrame({'location': list(temp['location']), 'Wait': list_W, 'Now': list_N})
    result['Wait_percentage'] = result['Wait'] / 1000
    result['Now_percentage'] = result['Now'] / 1000
    # print(result)

Please input the length of the grid:
5
Please input the width of the grid:
5
Please input how many orders are waiting before this order:
4
Please input previous order location:example:1,1
1,1


In [129]:
result2=result

In [116]:
result1.head()

,location,Wait,Now,Wait_percentage,Now_percentage
0,"(0, 0)",883,117,0.883,0.117
1,"(0, 1)",895,105,0.895,0.105
2,"(0, 2)",568,432,0.568,0.432
3,"(0, 3)",519,481,0.519,0.481
4,"(0, 4)",367,633,0.367,0.633


In [118]:
result0.head()

,location,Wait,Now,Wait_percentage,Now_percentage
0,"(0, 0)",1000,0,1.000,0.000
1,"(0, 1)",987,13,0.987,0.013
2,"(0, 2)",908,92,0.908,0.092
3,"(0, 3)",880,120,0.880,0.120
4,"(0, 4)",880,120,0.880,0.120


In [130]:
result2.head()

,location,Wait,Now,Wait_percentage,Now_percentage
0,"(0, 0)",834,166,0.834,0.166
1,"(0, 1)",686,314,0.686,0.314
2,"(0, 2)",632,368,0.632,0.368
3,"(0, 3)",615,385,0.615,0.385
4,"(0, 4)",612,388,0.612,0.388


In [126]:
result3.head()

,location,Wait,Now,Wait_percentage,Now_percentage
0,"(0, 0)",752,248,0.752,0.248
1,"(0, 1)",770,230,0.770,0.230
2,"(0, 2)",545,455,0.545,0.455
3,"(0, 3)",375,625,0.375,0.625
4,"(0, 4)",104,896,0.104,0.896


In [133]:
array=np.asarray(result['Wait_percentage'].tolist())
array=array.reshape(5,5)

col_sc=bqplot.ColorScale(scheme='Reds')
x_sc=bqplot.OrdinalScale()#value can be ordered
y_sc=bqplot.OrdinalScale()


x_ax=bqplot.Axis(scale=x_sc)
y_ax=bqplot.Axis(scale=y_sc,orientation='vertical')
c_ax=bqplot.ColorAxis(scale=col_sc,orientation='vertical',side='right')

heat_map=bqplot.GridHeatMap(color=array,
                            scales={'color':col_sc,
                                    'row':y_sc,
                                    'column':x_sc},
                           )
fig=bqplot.Figure(marks=[heat_map],axes=[x_ax,y_ax,c_ax])
display(fig)

Figure(axes=[Axis(scale=OrdinalScale()), Axis(orientation='vertical', scale=OrdinalScale()), ColorAxis(orienta…

In [86]:
@ipywidgets.interact(queue_count=(0,5,1))
def mapplot(queue_count=0):
    previous_order_location=(1,1)
    for repeat in list(range(1000)):
        new_map=real_map(generate_map)
        for nodes in list(new_map.nodes()):
            for location, decision in judgement(new_map, nodes,queue_count,previous_order_location).items():
                location_list.append(location)
                decision_list.append(decision)
    result = pd.DataFrame({'location': location_list, 'decision': decision_list})
    temp = result.groupby('location')['decision'].sum().reset_index()
    list_N = []
    list_W = []
    for i in list(range(length * width)):
        n_N = temp['decision'][i].count('N')
        n_W = temp['decision'][i].count('W')
        list_N.append(n_N)
        list_W.append(n_W)
    result = pd.DataFrame({'location': list(temp['location']), 'Wait': list_W, 'Now': list_N})
    result['Wait_percentage'] = result['Wait'] / 1000
    result['Now_percentage'] = result['Now'] / 1000
    array=np.asarray(result['Wait_percentage'].tolist())
    array=array.reshape(5,5)
    col_sc=bqplot.ColorScale(scheme='Reds')
    x_sc=bqplot.OrdinalScale()#value can be ordered
    y_sc=bqplot.OrdinalScale()

    x_ax=bqplot.Axis(scale=x_sc)
    y_ax=bqplot.Axis(scale=y_sc,orientation='vertical')
    c_ax=bqplot.ColorAxis(scale=col_sc,orientation='vertical',side='right')

    heat_map=bqplot.GridHeatMap(color=array,
                                scales={'color':col_sc,
                                        'row':y_sc,
                                        'column':x_sc},
                               )
    fig=bqplot.Figure(marks=[heat_map],axes=[x_ax,y_ax,c_ax])
    display(fig)

interactive(children=(IntSlider(value=0, description='queue_count', max=5), Output()), _dom_classes=('widget-i…